## QA flow process

In this notebook we will implement flow for question-answering based on summaries.

The flow will look as follows:
- Load FAISS index
- Transform question to an embedding

- Find top k closest neighbours of the question (k=20 for now)

Relevant sources=0

While not 3 relevant sources are found:
- Ask ChatGPT if the source is relevant to the topic
- If the source is relevant, ask ChatGPT to create an answer based on the summary and the whole transcript and add 1 to relevant sources counter.

If no relevant source was found:
- Return message "I couldn't find relevant podcast segments."

If 1 relevant source was found:
- return answer as it is.

If 2 or 3 sources were found:
- rephrase the final answer to take into the account the answer from each source.

In [16]:
import os
import time
import json
import pandas as pd
import faiss
import numpy as np
from enum import Enum
from dotenv import find_dotenv, load_dotenv

from langchain import LLMChain  
from langchain.chat_models import ChatOpenAI
from langchain.prompts.chat import (  
ChatPromptTemplate,  
SystemMessagePromptTemplate,  
HumanMessagePromptTemplate,  
)  
from langchain.embeddings import OpenAIEmbeddings
from sentence_transformers import SentenceTransformer

dotenv_path = find_dotenv()
load_dotenv(dotenv_path)
OPENAI_API_KEY = os.environ["OPENAI_API_KEY"]


### Templates

In [2]:
SYSTEM_CHECK_TEMPLATE = """You are a highly skilled and intelligent assistant. Your output will be used by the system for further processing so you must follow the task precisely. Don't provide any text that is not relevant to the task.

The task is to assess the relevance of the context to the user question. If the question is relevant to the context, please provide a highly actionable and easy to understand answer to the question. Base your answer only on the provided context. If there is any information related to the question in the context, you have to answer. If it's not relevant at all, start your answer with "Not relevant" and provide an explanation why it's not relevant."""


HUMAN_CHECK_TEMPLATE = '''
User question: "{question}"

Context: """{context}"""'''


SYSTEM_FINAL_ANSWER_TEMPLATE = """You are a highly skilled and intelligent question answering assistant that loves to help people optimize their performance and health. Your output will be shown to the end user who wants to get a highly actionable and easy to understand answer to their question. The input you will receive are answers to the question based on the most relevant resources found by the search engine. The input will contain from 1 to 3 answers.

Your task is to rephrase the answers into one, coherent final answer that can be shown to the end user. Make use of listicles (max 10 items) and actionable examples to increase the chance of remembering the advice. Remember that the answer should be highly actionable. This is the main objective as we want to make the users optimize their performance and health, and make their life easier."""


HUMAN_FINAL_ANSWER_TEMPLATE = '''
User question: "{question}"

Answers: """{context}"""'''

### Functions

In [17]:
def segment_check_and_answer(question, context, model="gpt-3.5-turbo"):
    chat = ChatOpenAI(temperature=0, model_name=model)
  
    system_message_prompt = SystemMessagePromptTemplate.from_template(SYSTEM_CHECK_TEMPLATE)  
    human_message_prompt = HumanMessagePromptTemplate.from_template(HUMAN_CHECK_TEMPLATE)  
    chat_prompt = ChatPromptTemplate.from_messages([system_message_prompt, human_message_prompt])  
    
    chain = LLMChain(llm=chat, prompt=chat_prompt)  
    output = chain.run(question=question, context=context)

    return output


def find_relevant_segments(df_summary, question, indices, n_relevant=3):
    n_relevant_summaries = 0
    relevant_summaries = {}
    non_relevant_summaries = {}

    for ind in indices[0]:
        if n_relevant_summaries < n_relevant:
            context = df_summary.loc[ind, "summary"]
            answer = segment_check_and_answer(question=question, context=context)
            if answer.startswith("Not relevant"):
                non_relevant_summaries[int(ind)] = answer
            else:
                n_relevant_summaries+=1
                relevant_summaries[int(ind)] = {"answer": answer,
                                           "URL": df_summary.loc[ind, "url"],
                                           "keywords": df_summary.loc[ind, "keywords"]}
        else:
            return relevant_summaries, non_relevant_summaries
    return relevant_summaries, non_relevant_summaries

def final_answer(question, summaries, model="gpt-3.5-turbo"):
    chat = ChatOpenAI(temperature=0, model_name=model)
  
    system_message_prompt = SystemMessagePromptTemplate.from_template(SYSTEM_FINAL_ANSWER_TEMPLATE)  
    human_message_prompt = HumanMessagePromptTemplate.from_template(HUMAN_FINAL_ANSWER_TEMPLATE)  
    chat_prompt = ChatPromptTemplate.from_messages([system_message_prompt, human_message_prompt])  

    answers = "\n".join([f"ANSWER {i+1}:\n{answer['answer']}\n" for i, answer in enumerate(summaries.values())])
    
    chain = LLMChain(llm=chat, prompt=chat_prompt) 
    output = chain.run(question=question, context=answers)

    return output

def qa_full_flow(question, df_summary, embedding_model, index, n_neighbours=20, n_relevant_summaries=3):
    start_time = time.time()
    
    # Encoding question to embedding space
    if str(type(embedding_model)) == "<class 'sentence_transformers.SentenceTransformer.SentenceTransformer'>":
        n_dim = embedding_model.get_sentence_embedding_dimension()
        emb_query = embedding_model.encode(question)
    elif str(type(embedding_model)) == "<class 'langchain.embeddings.openai.OpenAIEmbeddings'>":
        n_dim = 1536
        emb_query = embedding_model.embed_query(question)
    else:
        raise TypeError("Wrong Embedding model")
    
    # Finding relevant summaries
    # Getting indices
    distances, indices = index.search(np.array(emb_query).reshape(1, n_dim), n_neighbours)
    # Getting answers from segments
    relevant_summaries, non_relevant_summaries = find_relevant_segments(df_summary, question, indices, n_relevant_summaries)
    json.dump(relevant_summaries, open(f"data/qa-outputs/{question}-relevant.json", "w"))
    json.dump(non_relevant_summaries, open(f"data/qa-outputs/{question}-non-relevant.json", "w"))

    # Getting the final answer

    answer = final_answer(question=question, summaries=relevant_summaries)

    
    answer += "\n\nHere are HubermanLab Podcast segments that relate to your question:\n\n"
    for i, value in enumerate(relevant_summaries.values()):
        answer += f'{i+1}. {value["URL"]}\n'
        
    with open(f"data/qa-outputs/{question}-final-answer.txt", "w") as f:
        f.write(answer)

    end_time = time.time()
    print(f"Final time: {round(end_time-start_time, 2)}")
    return answer


### Embedding model and index

In [5]:
class EmbeddingModel(Enum):
    SBERT = "sbert"
    OPENAI = "openai"

In [26]:
N_DIM = 384
N_NEIGHBOURS = 20
model = SentenceTransformer('all-MiniLM-L6-v2')
index = faiss.read_index("data/embeddings/faiss_summary_index_sbert.faiss")
df_summary = pd.read_csv(f"data/summary_kmeans_with_chatgpt_and_keywords_final.csv")

In [29]:
model1 = OpenAIEmbeddings()
isinstance(model, SentenceTransformer), isinstance(model1, OpenAIEmbeddings)

(True, True)

## Full flow benchmark
- What is the inference time
- What is the quality of outputs
- Is the "acceptance" model working correctly

INPUT: 
- question

OUTPUTS:
- accepted summaries
- disregarded summaries
- final answer


### Test of outputs - 1

In [114]:
queries = ["Tools for creating an effective training program.",
           "How to sleep well?",
           "How to cure depression?",
           "How to increase the chance of successfully conceiving a child?",
           "Tools for increasing focus."]

In [115]:
for query in queries:
    answer = qa_full_flow(query, df_summary, model, index, n_neighbours=20, n_relevant_summaries=3)
    print(answer)
    

Final time: 23.62
To create an effective training program, here are some actionable tools and strategies you can incorporate:

1. Nasal Breathing and Breath Holds: Incorporate nasal breathing and breath holds into your training program. This has been shown to have beneficial effects on physical and cognitive performance.

2. Assess Properly: Use tools or methods to properly evaluate your current fitness level. This will help you identify specific training goals and areas for improvement.

3. Set SMART Goals: Use the SMART system for goal setting. Set specific, measurable, attainable, realistic, and timely goals that align with your limitations and objectives.

4. Plan Structured Training: Utilize planning tools or templates to create a structured and progressive training program. This will ensure that your program is tailored to your needs and goals.

5. Incorporate Mental Training and Visualization: Consider incorporating mental training and visualization practices into your program. 

### Test of outputs - 2 (Twitter)

In [12]:
queries = ["What are the preferable day phases to expose myself to the sun and how does it affect sleep?",
           "What is the best diet for losing weight?",
           "What's the best way to stimulate muscle growth for strength?",
           "What's the impact of coffee and adenosine on our sleep?"
           ]

In [15]:
question = "What's the impact of coffee and adenosine on our sleep?"
answer = qa_full_flow(question, df_summary, model, index, n_neighbours=20, n_relevant_summaries=3)
print(answer)

Final time: 36.12
The impact of coffee and adenosine on our sleep is that caffeine, which is found in coffee, binds to adenosine receptors in our brain. Adenosine is a compound that accumulates throughout the day and makes us feel tired. When caffeine binds to adenosine receptors, it prevents adenosine from signaling tiredness, leading to increased alertness and a delay in the timing of sleepy signals. However, it's important to note that caffeine does not create more energy, it simply changes the timing of our sleep and wakefulness signals.

To optimize your sleep and maintain a healthy sleep schedule, here are some actionable tips:

1. Limit caffeine intake: Be mindful of your caffeine consumption, especially in the afternoon and evening. Consider switching to decaffeinated coffee or herbal tea in the later part of the day.

2. Establish a bedtime routine: Create a relaxing routine before bed to signal to your body that it's time to wind down. This can include activities such as read

In [16]:
for query in queries:
    answer = qa_full_flow(query, df_summary, model, index, n_neighbours=20, n_relevant_summaries=3)
    print(answer)
    